昨日は、lightGBMに「築年数」「面積」「間取り」「契約期間」「その部屋のある階数」「全体の階数」「所在地」「最寄り駅」「最寄り駅までの所要時間」をいれてみて、過学習に陥った。<br>
本日は正則化を試みる。

In [465]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb

訓練データの読み込み

In [466]:
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange_scores = pd.read_csv('room_arrange_scores.csv')
contract_span = pd.read_csv('contract_span.csv')
floor_scores = pd.read_csv('floor_scores.csv')
Floor_scores = pd.read_csv('capital_floor_scores.csv')
wards = pd.read_csv('wards.csv')
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')

rent = pd.read_csv('rent.csv')

テストデータの読み込み

In [467]:
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange_scores = pd.read_csv('test_room_arrange_scores.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_wards = pd.read_csv('test_wards.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')

訓練データの作成

In [468]:
X_train = pd.concat([house_age, area_size, room_arrange_scores, contract_span, floor_scores, Floor_scores, wards, stations, minits], axis=1)
y_train = rent

In [469]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

テストデータの作成

In [470]:
X_test = pd.concat([test_house_age, test_area_size, test_room_arrange_scores, test_contract_span, test_floor_scores, test_Floor_scores, test_wards, test_stations, test_minits], axis=1)

lightGBMで正則化を行う。

In [471]:
X_train = pd.concat([house_age, area_size, room_arrange_scores, contract_span, floor_scores, Floor_scores, wards, stations, minits], axis=1)
y_train = rent


X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_room_arrange_scores, test_contract_span, test_floor_scores, test_Floor_scores, test_wards, test_stations, test_minits], axis=1)

category_lists = ['市区町村','最寄り駅']

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'lambda_l2':0.1
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['市区町村', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
c:

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001257 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1089
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 9
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 37664.7	valid_1's rmse: 42917.2
[20]	training's rmse: 23384.3	valid_1's rmse: 31945.7
[30]	training's rmse: 18454.2	valid_1's rmse: 28175.6
[40]	training's rmse: 16188.9	valid_1's rmse: 26627.8
[50]	training's rmse: 14906.5	valid_1's rmse: 25657.9
[60]	training's rmse: 14146.9	valid_1's rmse: 25145.4
[70]	training's rmse: 13501.4	valid_1's rmse: 24684.2
[80]	training's rmse: 12969.2	valid_1's rmse: 24338.1
[90]	training's rmse: 12503.5	valid_1's rmse: 24038.6
[100]	training's rmse: 12124.2	valid_1's rmse: 23861.6
[110]	training's rmse: 11770.3	valid_1's rmse: 23

カテゴリ化がうまくいってない気がする・・・。<br>
カテゴリデータ意外のもののみでlightGBMを走らせる。

In [472]:
X_train = pd.concat([house_age, area_size, room_arrange_scores, contract_span, floor_scores, Floor_scores, minits], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_room_arrange_scores, test_contract_span, test_floor_scores, test_Floor_scores, test_minits], axis=1)

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'lambda_l2':0.00001
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000279 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 669
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 7
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 40592.2	valid_1's rmse: 45159.1
[20]	training's rmse: 31520	valid_1's rmse: 38616.1
[30]	training's rmse: 28630	valid_1's rmse: 36860.2
[40]	training's rmse: 27312	valid_1's rmse: 36076.3
[50]	training's rmse: 26453.4	valid_1's rmse: 35461.5
[60]	training's rmse: 25804.2	valid_1's rmse: 34880.8
[70]	training's rmse: 25218.1	valid_1's rmse: 34454.6
[80]	training's rmse: 24618.5	valid_1's rmse: 34120.1
[90]	training's rmse: 24051.9	valid_1's rmse: 33643.5
[100]	training's rmse: 23657.8	valid_1's rmse: 

スコアは下がってしまった。カテゴリ化データはあったほうがよさそう。

ほかの特徴量の組み合わせを試してみる。<br>
以下は、最寄り駅と市区町村の情報がかぶっているという観点から「市区町村」を消したもの。

In [492]:
X_train = pd.concat([house_age, area_size, room_arrange_scores, contract_span, floor_scores, Floor_scores, stations, minits], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_room_arrange_scores, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits], axis=1)

category_lists = ['最寄り駅']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'lambda_l2':0.0000001
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\k

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001141 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1065
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 8
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 37640.8	valid_1's rmse: 42926
[20]	training's rmse: 23618.4	valid_1's rmse: 32111.2
[30]	training's rmse: 18445.5	valid_1's rmse: 28114.2
[40]	training's rmse: 16246.6	valid_1's rmse: 26505.1
[50]	training's rmse: 15058.8	valid_1's rmse: 25724.1
[60]	training's rmse: 14231.8	valid_1's rmse: 25068.5
[70]	training's rmse: 13604	valid_1's rmse: 24536.4
[80]	training's rmse: 13048.7	valid_1's rmse: 24134.8
[90]	training's rmse: 12604.5	valid_1's rmse: 23857.5
[100]	training's rmse: 12238.4	valid_1's rmse: 23661
[110]	training's rmse: 11888.2	valid_1's rmse: 23479
[1

今までの中であれば悪くはないが、まだまだ過学習している。

各変数におかしな値が入ってしまっていないかチェックする

まずは訓練データについて

築年数

In [474]:
house_age.describe()

,築年数
count,31470.000000
mean,194.336543
std,181.463280
min,0.000000
25%,41.000000
50%,161.000000
75%,329.000000
max,12235.000000


↑最大値はあきらかにおかしい。

面積

In [475]:
area_size.describe()

,面積
count,31470.000000
mean,34.808648
std,18.814245
min,4.860000
25%,22.440000
50%,28.410000
75%,42.390000
max,430.100000


間取り点数

In [476]:
room_arrange_scores.describe()

,間取り得点
count,31470.000000
mean,2.874579
std,1.462443
min,1.000000
25%,2.000000
50%,2.000000
75%,4.000000
max,9.000000


契約期間

In [477]:
contract_span.describe()

,契約期間
count,31366.000000
mean,1.579295
std,0.920114
min,0.000000
25%,2.000000
50%,2.000000
75%,2.000000
max,10.000000


所在階

In [478]:
floor_scores.describe()

,所在階
count,30756.000000
mean,3.973599
std,3.954655
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,53.000000


全体の階数

In [479]:
Floor_scores.describe()

,全体の階数
count,30756.000000
mean,6.976427
std,6.580764
min,1.000000
25%,3.000000
50%,5.000000
75%,10.000000
max,58.000000


市区町村のカテゴリ

In [480]:
wards.describe()

,市区町村
count,31262.000000
mean,12.480871
std,6.415315
min,1.000000
25%,7.000000
50%,12.000000
75%,19.000000
max,23.000000


最寄り駅のカテゴリ

In [481]:
stations.describe()

,最寄り駅
count,31470.000000
mean,153.743248
std,109.301518
min,1.000000
25%,64.000000
50%,125.000000
75%,235.000000
max,463.000000


所要時間のカテゴリ

In [482]:
minits.describe()

,所要時間
count,31223.000000
mean,7.589661
std,4.558802
min,1.000000
25%,4.000000
50%,7.000000
75%,10.000000
max,42.000000


次にテストデータについて

築年数

In [483]:
test_house_age.describe()

,築年数
count,31262.000000
mean,195.390026
std,165.242012
min,0.000000
25%,43.000000
50%,161.000000
75%,330.000000
max,1336.000000


面積

In [484]:
test_area_size.describe()

,面積
count,31262.000000
mean,34.861260
std,18.904095
min,1.000000
25%,22.280000
50%,28.490000
75%,42.600000
max,407.610000


間取りの得点

In [485]:
test_room_arrange_scores.describe()

,間取り得点
count,31262.000000
mean,2.885644
std,1.477383
min,1.000000
25%,2.000000
50%,2.000000
75%,4.000000
max,11.000000


契約期間

In [486]:
test_contract_span.describe()

,契約期間
count,31175.000000
mean,1.585560
std,0.912479
min,0.000000
25%,2.000000
50%,2.000000
75%,2.000000
max,7.000000


所在階

In [487]:
test_floor_scores.describe()

,所在階
count,30493.000000
mean,3.993015
std,3.962162
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,53.000000


全体の階数

In [488]:
test_Floor_scores.describe()

,全体の階数
count,30493.000000
mean,6.979143
std,6.544910
min,1.000000
25%,3.000000
50%,5.000000
75%,10.000000
max,60.000000


市区町村のカテゴリ

In [489]:
test_wards.describe()

,市区町村
count,31262.000000
mean,10.933689
std,6.590213
min,1.000000
25%,5.000000
50%,10.000000
75%,16.000000
max,23.000000


最寄り駅のカテゴリ

In [490]:
test_stations.describe()

,最寄り駅
count,31262.000000
mean,156.006302
std,110.946393
min,1.000000
25%,65.000000
50%,126.500000
75%,240.000000
max,475.000000


所要時間

In [491]:
test_minits.describe()

,所要時間
count,31019.000000
mean,7.557884
std,4.549209
min,1.000000
25%,4.000000
50%,7.000000
75%,10.000000
max,49.000000
